In [45]:
from naver import get_stock
import numpy as np
import pymysql
from dbconnect import connection

samsung = get_stock("005930", 2)
samsung.dtypes
samsung[['종가', '전일비', '시가', '고가', '저가', '거래량']] = \
samsung[['종가', '전일비', '시가', '고가', '저가', '거래량']].astype(np.int64)
samsung['거래금액'] = samsung['종가'] * samsung['거래량']


In [43]:
samsung.dtypes

날짜      object
종가       int64
전일비      int64
시가       int64
고가       int64
저가       int64
거래량      int64
거래금액     int64
dtype: object

In [46]:
cur, con = connection()

In [20]:
samsung['종목'] = '005930'
samsung = samsung[['종목', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '거래금액', ]]


In [23]:
type(samsung)

pandas.core.frame.DataFrame

In [24]:
for idx,row in samsung.iterrows():
    print(idx, row)

1 종목            005930
날짜        2023.02.07
종가             61900
전일비              300
시가             61900
고가             62500
저가             61600
거래량         14491039
거래금액    896995314100
Name: 1, dtype: object
2 종목            005930
날짜        2023.02.06
종가             61600
전일비             2200
시가             62800
고가             63000
저가             61600
거래량         15529356
거래금액    956608329600
Name: 2, dtype: object
3 종목            005930
날짜        2023.02.03
종가             63800
전일비              300
시가             63900
고가             64000
저가             63000
거래량         15194598
거래금액    969415352400
Name: 3, dtype: object
4 종목             005930
날짜         2023.02.02
종가              63500
전일비              1700
시가              63200
고가              63900
저가              62600
거래량          23285983
거래금액    1478659920500
Name: 4, dtype: object
5 종목             005930
날짜         2023.02.01
종가              61800
전일비               800
시가              62600
고가              62700
저

In [21]:
sql = "INSERT INTO stock_day VALUES (%s , %s , %s , %s , %s , %s , %s , %s , %s)"

for idx, row in samsung.iterrows():
    try:
        cur.execute(sql,list(row.values))
    except:
        pass
    
con.commit()


In [26]:
cur.execute("""
CREATE TABLE KRX(
    ISU_CD VARCHAR(200),
    ISU_SRT_CD VARCHAR(200),
    ISU_NM VARCHAR(200),
    ISU_ABBRV VARCHAR(200),
    ISU_ENG_NM VARCHAR(200),
    LIST_DD VARCHAR(200),
    MKT_TP_NM VARCHAR(200),
    SECUGRP_NM VARCHAR(200),
    SECT_TP_NM VARCHAR(200),
    KIND_STKCERT_TP_NM VARCHAR(200),
    PARVAL VARCHAR(200),
    LIST_SHRS VARCHAR(200),
    PRIMARY KEY(ISU_SRT_CD)
    );

""")

0

In [28]:
import requests
krx_url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"
payload = {"bld"  : "dbms/MDC/STAT/standard/MDCSTAT01901",
            "locale"  : "ko_KR",
            "mktId"  : "ALL",
            "share"  : "1",
            "csvxls_isNo"  : "false",}
r = requests.post(krx_url, data=payload)

rt = r.json()

sql = "INSERT INTO KRX VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

for data in rt['OutBlock_1']:
    try:
        cur.execute(sql, list(data.values()))
    except:
        pass


con.commit()


In [33]:
cur.execute("""
SELECT * 
FROM stock_day AS b
JOIN KRX AS c 
ON c.ISU_SRT_CD = b.symbol;
""")

int

In [32]:
cur.execute("""
select symbol
from stock_day
""")

10